In [1]:
import pandas as pd
import numpy as np
from neo4j import GraphDatabase

In [2]:
df = pd.read_csv('Datasets\dataset_final.csv')

In [3]:
uri = "neo4j+s://b073ebf5.databases.neo4j.io"
user = "neo4j"
password = "dFUBgoUZAXirgfzSyJT4z_dpznMWBU-g5Z53SHGtonE"

driver = GraphDatabase.driver(uri, auth=(user, password))

# Test de conexión
def test_connection():
    with driver.session() as session:
        result = session.run("RETURN 'Conexión exitosa con AuraDB' AS mensaje")
        for record in result:
            print(record["mensaje"])

test_connection()

Conexión exitosa con AuraDB


In [4]:
def crear_nodo_depto(tx, datos):
    query = """
    CREATE (:Departamento {
        id: $id,
        latitude: $latitude,
        longitude: $longitude
    })
    """
    tx.run(query, **datos)
    
def cargar_deptos(df):
    with driver.session() as session:
        for _, row in df.iterrows():
            datos = {
                "id": row["id"],
                "latitude": float(row["latitud"]),
                "longitude": float(row["longitud"])
            }
            session.write_transaction(crear_nodo_depto, datos)

In [5]:
cargar_deptos(df)

C:\Users\Pc-ADS\AppData\Local\Temp\ipykernel_28644\759094799.py:19: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(crear_nodo_depto, datos)
[#DA42]  _: <CONNECTION> error: Failed to write data to connection ResolvedIPv4Address(('34.121.155.65', 7687)) (ResolvedIPv4Address(('34.121.155.65', 7687))): ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)
Unable to retrieve routing information
Transaction failed and will be retried in 1.1121626863011316s (Unable to retrieve routing information)
[#DA43]  _: <CONNECTION> error: Failed to write data to connection IPv4Address(('si-b073ebf5-e88d.production-orch-0067.neo4j.io', 7687)) (ResolvedIPv4Address(('34.121.155.65', 7687))): ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)
Transaction failed and will be retried in 1.6355862283877145s (Failed to write data to connectio